In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.stats import linregress
from pathlib import Path
import matplotlib.pyplot as plt

# === SETTINGS ===
model_name = "EC-Earth3"

# Base CMIP6 path for this model
data_dir = Path(f"/dmidata/projects/nckf/cmip6/historical/{model_name}/")

# Output paths (your working directory)
output_dir = Path("/data/users/frekle/AMOC_analysis/results/")
output_dir.mkdir(parents=True, exist_ok=True)

output_csv = output_dir / f"AMOC_trends_{model_name}.csv"
plot_dir = output_dir / f"plots_AMOC_{model_name}"
plot_dir.mkdir(exist_ok=True)

# Variable and analysis parameters
var_name = "msftyz"
basin_index = 1        # Atlantic basin
depth_min = 500.0      # only consider depths > 500 m
target_lat = 26.0      # standard AMOC latitude
start_year, end_year = 1900, 2005

# === HELPER FUNCTIONS ===
def load_amoc_timeseries(file_path):
    """Load AMOC strength time series (Atlantic, max below 500 m at 26°N)."""
    ds = xr.open_dataset(file_path)
    da = ds[var_name]

    # --- Select Atlantic basin (basin=1) ---
    if "basin" in da.dims:
        da = da.sel(basin=basin_index)

    # --- Select latitude closest to 26°N and depth > 500 m ---
    da_sel = da.sel(rlat=target_lat, method="nearest").sel(lev=slice(depth_min, None))

    # --- Take maximum overturning strength below 500 m ---
    amoc_strength = da_sel.max(dim="lev")

    # --- Convert to annual mean and subset 1900–2005 ---
    amoc_annual = amoc_strength.resample(time="Y").mean()
    amoc_annual = amoc_annual.sel(time=slice(f"{start_year}-01-01", f"{end_year}-12-31"))

    return amoc_annual

def compute_trend(amoc_annual):
    """Compute linear trend in Sv/century and return slope and fit line."""
    years = amoc_annual["time"].dt.year.values
    values = amoc_annual.values
    mask = np.isfinite(values)
    years, values = years[mask], values[mask]
    slope, intercept, r, p, stderr = linregress(years, values)
    slope_sv_century = slope * 100 / 1e9  # kg/s → Sv/century
    fit = intercept + slope * years
    return slope_sv_century, years, values, fit

# === MAIN LOOP ===
results = []

# Find all msftyz files inside any realization folder
nc_files = sorted(data_dir.rglob("Omon/msftyz/*.nc"))

if not nc_files:
    print(f"⚠️ No files found for {model_name} in {data_dir}")
else:
    for nc_file in nc_files:
        try:
            print(f"Processing: {nc_file}")
            realization = nc_file.name.split("_")[4]  # e.g. 'r1i1p1f1'
            amoc_annual = load_amoc_timeseries(nc_file)
            trend, years, values, fit = compute_trend(amoc_annual)
            group = "AMOC-" if trend < 0 else "AMOC+"
            results.append({
                "Model": model_name,
                "Member": realization,
                "AMOC_trend_Sv_per_century": trend,
                "Group": group
            })
            print(f" → {realization}: {trend:.2f} Sv/century ({group})")

            # --- Plot AMOC time series ---
            plt.figure(figsize=(7, 4))
            plt.plot(years, values / 1e9, label="AMOC strength", lw=1.5)
            plt.plot(years, fit / 1e9, "--", color="red", label="Linear trend")
            plt.title(f"{model_name} {realization} | Atlantic basin (basin={basin_index})\n"
                      f"AMOC (max below 500m, {target_lat}°N)\nTrend = {trend:.2f} Sv/century")
            plt.xlabel("Year")
            plt.ylabel("AMOC Strength [Sv]")
            plt.grid(alpha=0.3)
            plt.legend()
            plt.tight_layout()
            plt.savefig(plot_dir / f"{model_name}_{realization}_AMOC_timeseries.png", dpi=150)
            plt.close()

        except Exception as e:
            print(f"⚠️ Skipping {nc_file.name}: {e}")

# === SAVE RESULTS ===
if results:
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Saved results to {output_csv}")
    print(df)

    # --- Ensemble summary ---
    mean_trend = df["AMOC_trend_Sv_per_century"].mean()
    std_trend = df["AMOC_trend_Sv_per_century"].std()
    n_members = len(df)
    print(f"\n📊 Ensemble summary for {model_name}:")
    print(f" → Members analyzed: {n_members}")
    print(f" → Mean AMOC trend: {mean_trend:.2f} ± {std_trend:.2f} Sv/century")

    print(f"\n✅ Saved {n_members} plots to {plot_dir}/")
else:
    print("❌ No valid results were generated.")


In [ ]:
# FPI analysis script

import xarray as xr
import numpy as np
import pandas as pd
from scipy.stats import linregress
from pathlib import Path
import matplotlib.pyplot as plt
import re

# === SETTINGS ===
model_name = "EC-Earth3"
base_path = Path(f"/dmidata/projects/nckf/cmip6/historical/{model_name}/")
output_dir = Path("/data/users/frekle/AMOC_analysis/results/FPI_original_test/")
output_dir.mkdir(parents=True, exist_ok=True)

# Regions from Li & Liu (2025)
regions = {
    "subpolar": {"lat": (46, 58), "lon": (-49, -21)},
    "gulfstream": {"lat": (41, 45), "lon": (-66, -40)},
}

start_year, end_year = 1900, 2005

# --- helper: discover coordinate names
def get_lat_lon_names(da):
    for lat_name in ["lat", "latitude", "y"]:
        if lat_name in da.dims or lat_name in da.coords:
            break
    else:
        raise ValueError("Could not find a latitude coordinate name.")

    for lon_name in ["lon", "longitude", "x"]:
        if lon_name in da.dims or lon_name in da.coords:
            break
    else:
        raise ValueError("Could not find a longitude coordinate name.")

    return lat_name, lon_name

# --- helper: region mean with cosine-lat weighting (0–360 or -180–180)
def select_region_mean(da, lat, lon):
    """
    Return area-weighted mean over a geographic box for both 1D and 2D lat/lon.
    Uses cos(lat) weights. Handles 0–360 and -180–180 longitudes, and wrap-around boxes.
    """
    lat_name, lon_name = get_lat_lon_names(da)

    # Coordinates
    LAT = da[lat_name]
    LON = da[lon_name]

    # Normalize longitudes to dataset convention
    lon_min, lon_max = lon
    if LON.max() > 180:   # dataset uses 0..360
        lon_min = lon_min % 360
        lon_max = lon_max % 360

    # Build region mask (works for 1D or 2D lat/lon)
    if LON.ndim == 1 and LAT.ndim == 1:
        # 1D case: build index masks then slice
        lat_mask = (LAT >= lat[0]) & (LAT <= lat[1])

        if lon_min <= lon_max:
            lon_mask = (LON >= lon_min) & (LON <= lon_max)
        else:
            # region crosses the dateline in 0..360 convention
            lon_mask = (LON >= lon_min) | (LON <= lon_max)

        sub = da.sel({lat_name: LAT[lat_mask], lon_name: LON[lon_mask]})

        # weights: cos(lat) expanded to 2D to match sub
        w_lat = xr.ufuncs.cos(np.deg2rad(sub[lat_name]))
        W = w_lat.broadcast_like(sub)

    else:
        # 2D/curvilinear case: build boolean mask directly on coordinates
        if lon_min <= lon_max:
            lon_ok = (LON >= lon_min) & (LON <= lon_max)
        else:
            lon_ok = (LON >= lon_min) | (LON <= lon_max)
        lat_ok = (LAT >= lat[0]) & (LAT <= lat[1])
        mask = lon_ok & lat_ok

        sub = da.where(mask)
        W = xr.ufuncs.cos(np.deg2rad(LAT)).where(mask)

    # Apply mask to weights where data is missing
    W = W.where(np.isfinite(sub))

    # Spatial dims = all non-time dims present in sub
    spatial_dims = [d for d in sub.dims if d != "time"]

    # Weighted mean over space (keep time)
    num = (sub * W).sum(dim=spatial_dims, skipna=True)
    den = W.sum(dim=spatial_dims, skipna=True)
    return num / den


# --- helper: load & merge all monthly files for one realization
def open_member_mfds(var_name, member):
    # e.g.: /.../r1i1p1f1/Omon/tos/*.nc
    files_dir = base_path / member / "Omon" / var_name
    files = sorted(files_dir.glob("*.nc"))   # FIXED: give a pattern
    if not files:
        return None
    # Let xarray align by coords; you can add chunks={} if desired
    ds = xr.open_mfdataset(files, combine="by_coords")
    return ds

# --- compute FPI time series and trend for a given var and member
def compute_FPI_for_member(var_name, pretty_name, units, member):
    ds = open_member_mfds(var_name, member)
    if ds is None:
        return None

    da = ds[var_name]

    # Ensure latitude is increasing if it's a dimension
    lat_name, lon_name = get_lat_lon_names(da)
    if lat_name in da.dims:
        da = da.sortby(lat_name)

    # subset in time & annual mean
    da = da.sel(time=slice(f"{start_year}-01-01", f"{end_year}-12-31"))
    if da.time.size == 0:
        return None

    da_ann = da.resample(time="YE").mean()  # 'YE' avoids FutureWarning

    # regional means
    subpolar = select_region_mean(da_ann, **regions["subpolar"])
    gulfstream = select_region_mean(da_ann, **regions["gulfstream"])

    fpi = (subpolar - gulfstream).rename("FPI")

    # linear trend (per century)
    years = fpi["time"].dt.year.values
    vals = fpi.values
    mask = np.isfinite(vals)
    years, vals = years[mask], vals[mask]
    slope, intercept, r, p, stderr = linregress(years, vals)
    trend_century = slope * 100

    return fpi, trend_century

# --- find all realizations available (e.g. r1i1p1f1, r131i1p1f1, etc.)
def list_realizations():
    mems = []
    for p in base_path.iterdir():
        if p.is_dir() and re.match(r"r\d+i\d+p\d+f\d+$", p.name):
            mems.append(p.name)
    return sorted(mems)

# === MAIN ===
rows = []
members = list_realizations()

# --- TEST ON ONE MEMBER ONLY ---
test_member = "r1i1p1f1"   # change if you want a different one
members = [m for m in members if m == test_member]
print(f"Testing only: {members}")

if not members:
    print(f"⚠️ No realization folders found under {base_path}")

for member in members:
    print(f"\nProcessing member: {member}")

    for var_name, pretty_name, units in [("tos", "SST", "°C"), ("sos", "SSS", "psu")]:
        try:
            result = compute_FPI_for_member(var_name, pretty_name, units, member)
            if result is None:
                print(f"  ⚠️ {pretty_name}: no data/time for {member}, skipping.")
                continue

            fpi, trend = result
            rows.append({
                "Model": model_name,
                "Member": member,
                "Variable": pretty_name,
                f"FPI_trend_{pretty_name}_per_century": trend
            })

            # plot
            plt.figure(figsize=(7, 4))
            plt.plot(fpi["time"].dt.year, fpi, label=f"{pretty_name} FPI", lw=1.5)
            plt.title(
                f"{model_name} {member} {pretty_name} FPI "
                f"(1900–2005)\nTrend = {trend:.3f} {units}/century"
            )
            plt.xlabel("Year")
            plt.ylabel(f"FPI ({units})")
            plt.grid(alpha=0.3)
            plt.legend()
            plt.tight_layout()
            plt.savefig(output_dir / f"{model_name}_{member}_FPI_{pretty_name}.png", dpi=150)
            plt.close()

            # save time series
            fpi.to_netcdf(output_dir / f"{model_name}_{member}_FPI_{pretty_name}.nc")

            print(f"  ✓ {pretty_name}: trend = {trend:.3f} {units}/century")

        except Exception as e:
            print(f"  ⚠️ Skipping {pretty_name} for {member}: {e}")

# save summary
if rows:
    df = pd.DataFrame(rows)
    csv_path = output_dir / f"FPI_trends_{model_name}.csv"
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Saved trends table: {csv_path}")
    print(df.head())
else:
    print("\n❌ No FPI results were generated.")


Testing only: ['r1i1p1f1']

Processing member: r1i1p1f1
  ✓ SST: trend = 0.917 °C/century
  ✓ SSS: trend = 0.624 psu/century

✅ Saved trends table: /data/users/frekle/EOF/AMOC_analysis/results/FPI_original/FPI_trends_EC-Earth3.csv
       Model    Member Variable  FPI_trend_SST_per_century  \
0  EC-Earth3  r1i1p1f1      SST                   0.916833   
1  EC-Earth3  r1i1p1f1      SSS                        NaN   

   FPI_trend_SSS_per_century  
0                        NaN  
1                    0.62352  
